In [1]:
import psycopg2 as ps
import pandas as pd

In [4]:
def connect_to_db(host_name, dbname, username, password, port):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print("Connected")
    return conn


Connected


In [2]:
host_name = 'youtube.ccotmgxsehhh.us-west-1.rds.amazonaws.com'
dbname = 'yt'
port = '5432'
username = 'avishek'
password = 'itsHello8920'

conn = connect_to_db(host_name, dbname, username, password, port)

In [ ]:
create_table_cmd = """cretae table if not exists videos"""